In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
TAXONOMIC_PATH = os.environ.get('TAXONOMIC_PATH')
K3_PATH = os.environ.get('K3_PATH')

In [3]:
print(TAXONOMIC_PATH)

None


In [4]:
y = pd.read_csv(TAXONOMIC_PATH, usecols=['Species'])
x = pd.read_csv(K3_PATH).drop("Unnamed: 0", axis=1)


ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
xx = pca.fit_transform(x)
pca.inverse_transform(xx)
pca.explained_variance_ratio_.sum()
print(f"{x.shape[1]} columns originally present, \n{pca.n_components_} account for 95 percent variance ")
print(f"corresponds to {100*pca.n_components_/x.shape[1]}% of total")

64 columns originally present, 
33 account for 95 percent variance 
corresponds to 51.5625% of total


In [27]:
train_x, test_x, train_y, test_y = train_test_split(xx, y, test_size=0.2, random_state=7)

In [28]:
train_y.describe()

,Species
count,10601
unique,2351
top,Escherichia coli
freq,712


In [29]:
train_x.shape

(10601, 33)

In [30]:
model = Sequential()
model.add(Dense(20, input_shape =(None, train_x.shape[1]) , activation="relu"))
model.add(Dense(len(y.Species.unique()),activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, None, 20)          680       
                                                                 
 dense_5 (Dense)             (None, None, 2621)        55041     
                                                                 
Total params: 55,721
Trainable params: 55,721
Non-trainable params: 0
_________________________________________________________________


In [32]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y)
test_y = lb.transform(test_y)
train_y = lb.transform(train_y)

In [33]:
model.fit(train_x, train_y, epochs = 25)

Epoch 1/25
332/332 [==============================] - 1s 3ms/step - loss: 5.2220 - accuracy: 0.2854
Epoch 2/25
332/332 [==============================] - 1s 3ms/step - loss: 2.6142 - accuracy: 0.5335
Epoch 3/25
332/332 [==============================] - 1s 3ms/step - loss: 1.6367 - accuracy: 0.6758
Epoch 4/25
332/332 [==============================] - 1s 3ms/step - loss: 1.0979 - accuracy: 0.7499
Epoch 5/25
332/332 [==============================] - 1s 3ms/step - loss: 0.7908 - accuracy: 0.7978
Epoch 6/25
332/332 [==============================] - 1s 3ms/step - loss: 0.6138 - accuracy: 0.8312
Epoch 7/25
332/332 [==============================] - 1s 3ms/step - loss: 0.5057 - accuracy: 0.8551
Epoch 8/25
332/332 [==============================] - 1s 3ms/step - loss: 0.4371 - accuracy: 0.8690
Epoch 9/25
332/332 [==============================] - 1s 3ms/step - loss: 0.3896 - accuracy: 0.8795
Epoch 10/25
332/332 [==============================] - 1s 3ms/step - loss: 0.3529 - accuracy: 0.8892

In [34]:
model.evaluate(test_x, test_y)

83/83 [==============================] - 0s 2ms/step - loss: 3.1197 - accuracy: 0.8155


[3.1196932792663574, 0.8155413269996643]

In [38]:
from sklearn import metrics
f1 = metrics.f1_score(lb.inverse_transform(test_y), lb.inverse_transform(model.predict(test_x)), average = 'weighted')
f1

0.7950534059712648

In [ ]:
i = 0
mismatch = 0
npredictions = 0
predictions = []
mismatch_species = []
mismatch_species_pred = []
y_tested = []
for y in test_y:
    a = model.predict(test_x[i:i+1]) 
    if a.max()>0.8:
        sp = lb.inverse_transform(np.array([y]))
        p_sp = lb.inverse_transform(a)
        if p_sp!=sp :
            mismatch_species.append(sp)
            mismatch_species_pred.append(p_sp)
            mismatch += 1
        npredictions += 1
        predictions.append(p_sp)
        y_tested.append(sp)
    i  += 1
print(mismatch)

98


In [ ]:
from sklearn import metrics
f1 = metrics.f1_score(y_tested, predictions, average = 'weighted')
f1

0.9247464198315868

In [ ]:
i, npredictions, mismatch

(2651, 1657, 98)

In [39]:
model.save("k3-net")

INFO:tensorflow:Assets written to: k3-net\assets
